In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import math
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from collections import Counter
from time import time

In [ ]:
TRAIN_TEXT_FILE_PATH = 'text.txt'
BATCH_SIZE = 16
epochs = 20


with open(TRAIN_TEXT_FILE_PATH) as text_file:
    text_sample = text_file.readlines()[::2]
text_sample = ' '.join(text_sample)

words = text_sample.split()
word_counts = Counter(words)
vocab = list(word_counts.keys())
vocab_size = len(vocab)
word_to_int = {word: i for i, word in enumerate(vocab)}
int_to_word = {i: word for word, i in word_to_int.items()}
SEQUENCE_LENGTH = 64
samples = [words[i:i+SEQUENCE_LENGTH+1] for i in range(len(words)-SEQUENCE_LENGTH)]
print(vocab)
print(word_to_int)
print(int_to_word)
print(vocab_size)

['Три', 'девицы', 'под', 'окном', '"Кабы', 'я', 'была', 'царица,-', 'То', 'на', 'весь', 'крещеный', 'мир', '-', 'бы', 'одна', 'Я', 'б', 'для', 'батюшки-царя', 'Только', 'вымолвить', 'успела,', 'И', 'в', 'светлицу', 'входит', 'царь,', 'Во', 'все', 'время', 'разговора', 'Речь', 'последней', 'по', 'всему', '"Здравствуй,', 'красная', 'девица,-', 'роди', 'богатыря', 'Вы', 'ж,', 'голубушки-сестрицы,', 'Поезжайте', 'вслед', 'за', 'мной,', 'Будь', 'из', 'вас', 'ткачиха,', 'В', 'сени', 'вышел', 'царь-отец.', 'Царь', 'недолго', 'собирался:', 'Салтан', 'пир', 'честной', 'А', 'потом', 'честные', 'гости', 'Положили', 'молодых', 'кухне', 'злится', 'повариха,', 'завидуют', 'оне', 'царица', 'молодая,', 'С', 'первой', 'ночи', 'понесла.', 'Салтан,', 'с', 'женой', 'простяся,', 'Ей', 'наказывал', 'себя', 'Между', 'тем,', 'как', 'он', 'далеко', 'Наступает', 'срок', 'родин;', 'над', 'ребенком,', 'Шлет', 'письмом', 'она', 'гонца,', 'ткачиха', 'поварихой,', 'Извести', 'ее', 'хотят,', 'Сами', 'шлют', 'гонца', 

In [ ]:
class TextDataset(Dataset):
    def __init__(self, samples, word_to_int):
        self.samples = samples
        self.word_to_int = word_to_int
    def __len__(self):
        return len(self.samples)
    def __getitem__(self, idx):
        sample = self.samples[idx]
        input_seq = torch.LongTensor([self.word_to_int[word] for word in sample[:-1]])
        target_seq = torch.LongTensor([self.word_to_int[word] for word in sample[1:]])
        return input_seq, target_seq

In [ ]:
dataset = TextDataset(samples, word_to_int)
dataloader = DataLoader(
    dataset,
    batch_size=BATCH_SIZE,
    shuffle=True,
)

In [ ]:
def generate_square_subsequent_mask(sz):
    """
    Generate a square mask for the sequence. The masked positions are filled with float('-inf').
    Unmasked positions are filled with float(0.0).
    """
    mask = (torch.triu(torch.ones(sz, sz)) == 1).transpose(0, 1)
    mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
    return mask


class PositionalEncoding(nn.Module):
    def __init__(self, max_len, d_model, dropout=0.1):
        """
        :param max_len: Input length sequence.
        :param d_model: Embedding dimension.
        :param dropout: Dropout value (default=0.1)
        """
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)
        self.register_buffer('pe', pe)
    def forward(self, x):
        """
        Inputs of forward function
        :param x: the sequence fed to the positional encoder model (required).
        Shape:
            x: [sequence length, batch size, embed dim]
            output: [sequence length, batch size, embed dim]
        """
        x = x + self.pe[:, :x.size(1)]
        return self.dropout(x)

In [ ]:
class TextGen(nn.Module):
    def __init__(self, vocab_size, embed_dim, num_layers, num_heads):
        super(TextGen, self).__init__()
        self.pos_encoder = PositionalEncoding(max_len=SEQUENCE_LENGTH, d_model=embed_dim)
        self.emb = nn.Embedding(vocab_size, embed_dim)
        self.decoder_layer = nn.TransformerDecoderLayer(
            d_model=embed_dim,
            nhead=num_heads,
            batch_first=True
        )
        self.decoder = nn.TransformerDecoder(
            decoder_layer=self.decoder_layer,
            num_layers=num_layers,
        )
        self.linear = nn.Linear(embed_dim, vocab_size)
        self.dropout = nn.Dropout(0.2)

    # Positional encoding is required. Else the model does not learn.
    def forward(self, x):
        emb = self.emb(x)

        # Generate input sequence mask with shape (SEQUENCE_LENGTH, SEQUENCE_LENGTH)
        input_mask = generate_square_subsequent_mask(x.size(1)).to(x.device)

        x = self.pos_encoder(emb)
        x = self.decoder(x, memory=x, tgt_mask=input_mask, memory_mask=input_mask)
        x = self.dropout(x)
        out = self.linear(x)
        return out

In [ ]:
learning_rate = 0.001
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = TextGen(
    vocab_size=vocab_size,
    embed_dim=20,
    num_layers=2,
    num_heads=2,
).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
# Total parameters and trainable parameters.
total_params = sum(p.numel() for p in model.parameters())
print(f"{total_params:,} total parameters.")
total_trainable_params = sum(
    p.numel() for p in model.parameters() if p.requires_grad)
print(f"{total_trainable_params:,} training parameters.\n")

612,708 total parameters.
612,708 training parameters.



In [ ]:
def train(model, epochs, dataloader, criterion):
    model.train()
    for epoch in range(epochs):
        running_loss = 0
        for input_seq, target_seq in dataloader:
            input_seq, target_seq = input_seq.to(device), target_seq.to(device)
            outputs = model(input_seq)
            target_seq = target_seq.contiguous().view(-1)
            outputs = outputs.view(-1, vocab_size)

            loss = criterion(outputs, target_seq.view(-1))

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            running_loss += loss.detach().cpu().numpy()
        epoch_loss = running_loss / len(dataloader)
        print(f"Epoch {epoch} loss: {epoch_loss:.3f}")

t1 = time()
train(model, epochs, dataloader, criterion)
print("Training time:", time() - t1)

Epoch 0 loss: 7.490
Epoch 1 loss: 5.333
Epoch 2 loss: 4.268
Epoch 3 loss: 3.736
Epoch 4 loss: 3.383
Epoch 5 loss: 3.122
Epoch 6 loss: 2.912
Epoch 7 loss: 2.741
Epoch 8 loss: 2.589
Epoch 9 loss: 2.457
Epoch 10 loss: 2.338
Epoch 11 loss: 2.236
Epoch 12 loss: 2.137
Epoch 13 loss: 2.051
Epoch 14 loss: 1.971
Epoch 15 loss: 1.901
Epoch 16 loss: 1.832
Epoch 17 loss: 1.773
Epoch 18 loss: 1.715
Epoch 19 loss: 1.661
Training time: 5791.59267783165


In [ ]:
def return_int_vector(text):
    words = text.split()
    input_seq = torch.LongTensor([word_to_int[word] for word in words[-SEQUENCE_LENGTH:]]).unsqueeze(0)
    return input_seq


def sample_next(predictions):
    """
    Greedy sampling.
    """
    # Greedy approach.
    probabilities = F.softmax(predictions[:, -1, :], dim=-1).cpu()
    next_token = torch.argmax(probabilities)
    return int(next_token.cpu())


def text_generator(sentence, generate_length):
    model.eval()
    sample = sentence
    for i in range(generate_length):
        int_vector = return_int_vector(sample)
        if len(int_vector) >= SEQUENCE_LENGTH - 1:
            break
        input_tensor = int_vector.to(device)
        with torch.no_grad():
            predictions = model(input_tensor)
        next_token = sample_next(predictions)
        sample += ' ' + int_to_word[next_token]
    print(sample)
    print('\n')

In [ ]:
sentences = [
    "Я помню чудное "
]
generate_length = 200
for sentence in sentences:
    print(f"PROMPT: {sentence}")
    text_generator(sentence, generate_length)

PROMPT: Я помню чудное 
Я помню чудное  мгновенье: Как совершенства образец. Всегда неправедно гонимый, И привлекательным лицом. Всегда восторженный герой И привлекательным лицом. Всегда восторженный герой И при конце последней части Добру достойный был венок. Мораль на нас наводит сон, И тихо край земли светлеет, И тихо край земли светлеет, И в Лете не потонет Быть может быть, такого пира; Да спокойно в свой удел Князь у синя моря ходит, Глядь - поверх текучих вод "Здравствуй, князь Гвидон ей отвечает: Диво б дивное хотел он уж в лесу, под елью белка; Белка песенки поет А орешки не простые, Ядра - чистый изумруд; Там у князя женка есть, Днем свет божий затмевает, Месяц под косой блестит, А орешки не простые, Ядра - чистый изумруд; Там у князя женка есть, Днем свет божий затмевает, Месяц под косой блестит, А сама-то величава, А сама-то величава, А сама-то величава, А сама-то величава, А сама-то величава, А сама-то величава, А сама-то величава, А как речь-то говорит, Молвить можно справ